<center style="border-radius:10px;
padding: 3rem 2rem;
border: 3px solid #B54257;
">
<h1 style="color:#F54257; 
font-size:3.0rem;
margin:0;
">KITTI Object Detection</h1>
<h2 style="color:#554257; 
font-size:2.0rem;
margin-top:1rem;
margin-bottom:2.5rem;
">YOLOv7 </h2>


# Steps to Follow
In this notebook, YOLO V7 model for object detection is used from https://github.com/WongKinYiu/yolov7.


<div style="width:100%;height:0;border-bottom: 3px solid #F03A4F;margin-bottom: 1rem;"></div>

Clone the YOLOV7 Repository:
- Obtain the YOLOv7 source code.

Acquire the YOLOv7 Model Weights:
- Secure the weight files for the YOLOv7 model.

Setting Up the Kitti Dataset:
- Retrieve the Kitti Dataset.
- Construct the YAML configuration file for the dataset.

Training the Model:
- Modify the format of yolov7-tiny.yaml.
- Run the train.py script.

Assessment:
- Perform COCO Evaluation on the testing data.
- Graphically represent MAP scores and outcomes.

# Clone The YOLOv7 Repository

In [ ]:
# Download YOLOv7
!git clone https://github.com/WongKinYiu/yolov7 # clone repo
%cd yolov7

# Install dependencies
%pip install -qr requirements.txt 

%cd ../
import torch
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

# Dowload the v7 Model

In [ ]:
!wget "https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-tiny.pt"

In [ ]:
!pwd


In [ ]:
! ls

# Preparing Dataset
<div style="width:100%;height:0;border-bottom: 3px solid #F03A4F;margin-bottom: 1rem;"></div>

In [ ]:
!pip install ultralytics -q

In [ ]:
from ultralytics import YOLO
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import json
import cv2

from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import shutil
from PIL import Image

In [ ]:
base_dir = Path('/kaggle/input/kitti-dataset')
img_path = base_dir / 'data_object_image_2' / 'training' / 'image_2'
label_path = Path('/kaggle/input/kitti-dataset-yolo-format/labels')
with open('/kaggle/input/kitti-dataset-yolo-format/classes.json','r') as f:
    classes = json.load(f)

print("Classes : {0}".format(classes))
ims = sorted(list(img_path.glob('*')))
labels = sorted(list(label_path.glob('*')))
pairs = list(zip(ims,labels))
train, test = train_test_split(pairs,test_size=0.1,shuffle=True)

print("Train and Test Size: {0} and {1}".format(len(train), len(test)))

In [ ]:
train_path = Path('train').resolve()
train_path.mkdir(exist_ok=True)
valid_path = Path('valid').resolve()
valid_path.mkdir(exist_ok=True)

In [ ]:
for t_img, t_lb in tqdm(train):
    im_path = train_path / t_img.name
    lb_path = train_path / t_lb.name
    shutil.copy(t_img,im_path)
    shutil.copy(t_lb,lb_path)

In [ ]:
for t_img, t_lb in tqdm(test):
    im_path = valid_path / t_img.name
    lb_path = valid_path / t_lb.name
    shutil.copy(t_img,im_path)
    shutil.copy(t_lb,lb_path)

# Preparing YAML
<div style="width:100%;height:0;border-bottom: 3px solid #F03A4F;margin-bottom: 1rem;"></div>

In [ ]:
yaml_file = 'names:\n'
yaml_file += '\n'.join(f'- {c}' for c in classes)
yaml_file += f'\nnc: {len(classes)}'
yaml_file += f'\ntrain: {str(train_path)}\nval: {str(valid_path)}'
with open('kitti.yaml','w') as f:
    f.write(yaml_file)

In [ ]:
!cat kitti.yaml

# Train the model
<div style="width:100%;height:0;border-bottom: 3px solid #F03A4F;margin-bottom: 1rem;"></div>

In [ ]:
%env WANDB_DISABLED=True

In [ ]:
# !wandb disabled
!python yolov7/train.py --batch-size 64 --img 640 640 --cfg /kaggle/working/yolov7/cfg/training/yolov7-tiny.yaml --epochs 20 --data ./kitti.yaml --weights yolov7-tiny.pt --device 0 --project 'yolov7-tiny' --name 'run1'

# Results
<div style="width:100%;height:0;border-bottom: 3px solid #F03A4F;margin-bottom: 1rem;"></div>

In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(Image.open('/kaggle/working/yolov7-tiny/run1/results.png'))
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(Image.open('/kaggle/working/yolov7-tiny/run1/confusion_matrix.png'))
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(15,80))
plt.imshow(Image.open('/kaggle/working/yolov7-tiny/run1/test_batch0_labels.jpg'))
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(15,80))
plt.imshow(Image.open('/kaggle/working/yolov7-tiny/run1/test_batch1_labels.jpg'))
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(15,80))
plt.imshow(Image.open('/kaggle/working/yolov7-tiny/run1/test_batch2_labels.jpg'))
plt.axis('off')
plt.show()